In [4]:
import csv
import pandas as pd
from apple_store_scraper import AppStore
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_never, wait_exponential, retry_if_exception_type
from requests.exceptions import RetryError

@retry(stop=stop_never, wait=wait_exponential(multiplier=1, max=600), retry=retry_if_exception_type(RetryError))
def get_reviews(app_name, country="cn"):
    app = AppStore(country=country, app_name=app_name)
    app.review(how_many=100000)
    return app.reviews

def save_reviews_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, mode='a', index=False)

def process_app(app_name):
    try:
        reviews = get_reviews(app_name)
        save_reviews_to_csv(reviews, f"{app_name}_reviews.csv")
    except Exception as e:
        print(f"Error processing app {app_name}: {e}")

def main():
    app_names = ["taobao"]  # Add your app names here
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_app, app_names)

if __name__ == "__main__":
    main()


2023-07-23 18:06:52,001 [INFO] Base - Searching for app id
2023-07-23 18:06:54,081 [INFO] Base - Initialised: AppStore('cn', 'taobao', 387682726)
2023-07-23 18:06:54,082 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/cn/app/taobao/id387682726
2023-07-23 18:06:59,269 [INFO] Base - [id:387682726] Fetched 160 reviews (160 fetched in total)
2023-07-23 18:07:05,449 [INFO] Base - [id:387682726] Fetched 360 reviews (360 fetched in total)
2023-07-23 18:07:11,598 [INFO] Base - [id:387682726] Fetched 560 reviews (560 fetched in total)
2023-07-23 18:07:17,737 [INFO] Base - [id:387682726] Fetched 760 reviews (760 fetched in total)
2023-07-23 18:07:23,473 [INFO] Base - [id:387682726] Fetched 940 reviews (940 fetched in total)
2023-07-23 18:07:29,616 [INFO] Base - [id:387682726] Fetched 1140 reviews (1140 fetched in total)
2023-07-23 18:07:35,658 [INFO] Base - [id:387682726] Fetched 1340 reviews (1340 fetched in total)
2023-07-23 18:07:41,766 [INFO] Base - [id:387682726] Fetched 1